[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# **Simulation example:** Python vs Cython vs C++ vs Julia

The code is adapted from the [blog post by The Multi-Agent AI Guy](https://medium.com/agents-and-robots/the-bitter-truth-python-3-11-vs-cython-vs-c-performance-for-simulations-babc85cdfef5). Big thank you Multi-Agent AI Guy!

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/hbFrPjeBpqg" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen>
</iframe>

Again let's activate julia

In [ ]:
from juliacall import Main as jl
%load_ext juliacall.ipython
# JuliaCall comes with its own Julia dependency file juliapkg.json
# however for binder it is much simpler to just reuse binder's installation mechanism
%julia Pkg.activate(Base.current_project())
%julia using PythonCall
%julia set_var(k, v) = @eval $(Symbol(k)) = $v

In [ ]:
import math
import random

from datetime import datetime
random.seed(datetime.now().timestamp())

WORLD_WIDTH = 2560
WORLD_HEIGHT = 1440
TIMESTEPS = 2000

jl.set_var("WORLD_WIDTH", WORLD_WIDTH)
jl.set_var("WORLD_HEIGHT", WORLD_HEIGHT)
jl.set_var("TIMESTEPS", TIMESTEPS)

# Python implementation

### Python definition of simulated Agent

In [ ]:
class PythonAgent():
    def __init__(self, x=None, y=None, world_width=0, world_height=0):
        super().__init__()

        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x if x else random.randint(0, self.world_width)
        self.y = y if y else random.randint(0, self.world_height)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # inital values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

    def update(self, food=()):
        self.age = self.age + 1

        # we can't move
        if self.vmax == 0:
            return

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x - self.target.x) ** 2 + (self.y - self.target.y) ** 2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a
            if min_dist < 100000:
                self.target = min_agent

        # initalize 'forces' to zero
        fx = 0
        fy = 0

        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05 * fx
        self.dy = self.dy + 0.05 * fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the world boundaries
        self.x = max(self.x, 0)
        self.x = min(self.x, self.world_width)
        self.y = max(self.y, 0)
        self.y = min(self.y, self.world_height)

In [ ]:
def main(Agent):

    class Predator(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.5

    class Prey(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.0

    class Plant(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 0.0

            
    # open the ouput file
    # f = open('output.csv', 'w')
    # print(0, ',', 'Title', ',', 'Predator Prey Relationship / Example 02 / Cython', file=f)

    # create initial agents
    kwargs = dict(
        world_width=WORLD_WIDTH,
        world_height=WORLD_HEIGHT,
    )
    preys = [Prey(**kwargs) for i in range(10)]
    predators = [Predator(**kwargs) for i in range(10)]
    plants = [Plant(**kwargs) for i in range(100)]

    timestep = 0
    while timestep < TIMESTEPS:
        # update all agents
        # no need to update the plants; they do not move
        for a in preys:
            a.update(plants)
        
        for a in predators:
            a.update(preys)

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant(**kwargs) for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep = timestep + 1

    print(len(predators), len(preys), len(plants))

### Cython definition of Agent

Cython has the unique advantage that you can easily reuse classes within Python

The Cython package was already compiled and is available as `agent` module

In [ ]:
from agent import Agent as CythonAgent

### Benchmark Python against Cython class

In [ ]:
%timeit -r 3 main(PythonAgent)

In [ ]:
%timeit main(CythonAgent)

# Full language switch

### Cython

The Cython module `agent` also includes a redefinition of the `main` routine, which gives a little extra speedup.

In [ ]:
import agent
%timeit agent.main(WORLD_WIDTH=WORLD_WIDTH, WORLD_HEIGHT=WORLD_HEIGHT, TIMESTEPS=TIMESTEPS)

## C++

The blog post also came with a C++ implementation. Only a small bug was fixed, so that C++ and Python version have identical simulations.

C++ needs be compiled first

In [ ]:
!g++ agent.cpp -o agent -std=c++11 -O3

then we can time it from the commandline

In [ ]:
!time ./agent $TIMESTEPS

and from python

In [ ]:
import subprocess
%timeit subprocess.run(["./agent", str(TIMESTEPS)], shell=True)

In [ ]:
import subprocess
%timeit subprocess.run(["./agent", str(TIMESTEPS)])

## Julia

Julia is not Object Oriented like Python, hence we need to restructure the class hierarchy.

Luckily it is super simple, as Predator, Prey and Plant don't really make use of Object Orientation inheritance anyway.

--------
#### 💻 your space
- 👉 DELETE ALL the following julia code
- 👉 build the julia version yourself (🙂 you can always ask for help 🙂)
- start untyped and finally add types (only needed for the `struct` fields)

In [ ]:
%%julia
# your space

In [ ]:
%%julia
# Base.@kwdef is a nice little helper which enables default arguments and construction by keywords
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

In [ ]:
%%julia
function update!(self, food)
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

In [ ]:
%%julia
function main(; show=true, world_width=WORLD_WIDTH, world_height=WORLD_HEIGHT, timesteps=TIMESTEPS)
    kwargs = (; world_width, world_height)
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < timesteps
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    show && println("$(length(predators)), $(length(preys)), $(length(plants))")
end

👉 delete up to this line

----------

### Benchmark the performance of you julia code

Try both `BenchmarkTools.@btime` and `BenchmarkTools.@benchmark`

In [ ]:
%julia using BenchmarkTools
%julia @btime main()

In order to compare C++ and Julia in more detail, it is informative to see the amount of garbage collection in Julia. (Because C++ does not have a garbage collector).

In [ ]:
%julia @benchmark main(show=false)

# Further resources

- [learning Julia](https://julialang.org/learning/)
- [juliacall / PythonCall.jl](https://cjdoris.github.io/PythonCall.jl/stable/) - my recommended interface to access Julia from Python
- [pyjulia / PyCall.jl](https://github.com/JuliaPy/PyCall.jl) - older interface, but from julia it has the lovely `py""` string to create python objects
- [julia-numpy / TyPython.jl](https://github.com/Suzhou-Tongyuan/jnumpy) - implement a python package in Julia (think of this more like Cython pyx files)
- [PackageCompiler.jl](https://julialang.github.io/PackageCompiler.jl/dev/) - create C libraries which you then can load from python

# Thank you for participating ❤️

You can always reach me at stephan.sahm@jolin.io or <a style="justify-content: center; padding: 7px; text-align: center; outline: none; text-decoration: none !important; color: #ffffff !important; width: 200px; height: 32px;border-radius: 16px; background-color: #0A66C2;" href="https://www.linkedin.com/company/jolin-io/" target="_blank">Follow on LinkedIn</a>
<!-- alternative follow https://www.linkedin.com/comm/mynetwork/discovery-see-all?usecase=PEOPLE_FOLLOWS&followMember=stephan-sahm-918656b7 -->

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>